In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from aqua.diagnostics.lat_lon_profiles import LatLonProfiles

In [3]:
dataset_dict = {'catalog': 'climatedt-phase1',
                'model': 'ICON',
                'exp': 'historical-1990',
                'source': 'lra-r100-monthly'}
common_dict = {'startdate': '1990-01-01',
               'enddate': '1999-12-31',
               'loglevel': 'info'}

In [4]:
lonlat_dataset = LatLonProfiles(**dataset_dict, **common_dict)

2025-06-06 18:44:05 :: LatLonProfiles :: INFO     -> No region provided, using lon_limits: None, lat_limits: None


In [5]:
run_dict = {'var': '2t',
            'units': 'degC'}

In [6]:
lonlat_dataset.run(**run_dict)

2025-06-06 18:44:05 :: LatLonProfiles :: INFO     -> Running LatLonProfiles for 2t
2025-06-06 18:44:05 :: ConfigPath :: INFO     -> ICON_historical-1990_lra-r100-monthly triplet found in in climatedt-phase1!


2025-06-06 18:44:05 :: Fixer :: INFO     -> Convention dictionary: eccodes-2.39.0
2025-06-06 18:44:05 :: Fixer :: INFO     -> Fix names in metadata is icon-destine-v1-lra
2025-06-06 18:44:05 :: Fixer :: INFO     -> Fix names icon-destine-v1-lra found in fixes files
2025-06-06 18:44:06 :: Reader :: INFO     -> Grid metadata is lon-lat
2025-06-06 18:44:06 :: Regridder :: WARNING  -> Source grid path not found. Please provide a dataset.
2025-06-06 18:44:06 :: Reader :: WARNING  -> Issues in the Regridder() init: trying with data
2025-06-06 18:44:24 | smmregrid.GridInspect |     INFO -> Updating the log_level to INFO
2025-06-06 18:44:24 :: Regridder :: INFO     -> Using provided dataset as a grid path for 2d
2025-06-06 18:44:24 :: Regridder :: INFO     -> Grid name: lon-lat
2025-06-06 18:44:24 :: Regridder :: INFO     -> Loading existing source area from /pfs/lustrep3/appl/local/climatedt/data/AQUA/areas/cell_area_ICON_historical-1990_lra-r100-monthly.nc.
2025-06-06 18:44:24 :: CoordTransa

In [7]:
lonlat_dataset